# Traps of using pandas

In [87]:
import pandas as pd
import numpy as np
import time

#### Dates

In [106]:
# Datetime format is always consider to be index...
list_of_dates = ['03/01/2021', '01/14/2021', '01/05/2021']
date_series = pd.to_datetime(list_of_dates)
date_series

DatetimeIndex(['2021-03-01', '2021-01-14', '2021-01-05'], dtype='datetime64[ns]', freq=None)

#### Memory of Pandas

In [2]:
# By default, Pandas always assigns the highest memory datatype to columns. This can be a problem when working with large datasets.
df = pd.DataFrame({'A': [1, 2, 3, 4, 5]})
df.dtypes

A    int64
dtype: object

In [3]:
df.A.memory_usage()

172

In [4]:
df['A'] = df['A'].astype('int8')
df.A.memory_usage()

137

In [5]:
datalake_df = pd.read_csv('from_datalake.csv')

In [6]:
datalake_df

,date,product,source,spends,factor,received_at
0,2024-01-01,product_1,source_1_product_1,1476304,0.323865,1733098377843
1,2023-02-01,product_1,source_2_product_1,2465434,0.336918,1733098377844
2,2024-07-01,product_1,source_1_product_1,1013569,0.439816,1733098377843
3,2024-03-01,product_1,source_1_product_1,1685347,0.292835,1733098377843
4,2024-05-01,product_1,source_2_product_1,4049280,0.245055,1733098377844
...,...,...,...,...,...,...
520,2024-06-01,product_2,source_4_product_3,65824,0.171763,1733098378012
521,2024-09-01,product_2,source_6_product_3,1451735,0.174637,1733098378011
522,2024-11-01,product_2,source_6_product_3,33171,0.256530,1733098378011
523,2023-02-01,product_2,source_4_product_3,75732,0.177700,1733098378012


In [51]:
def apply_func(row):
    return row.spends + row.factor

new_df = datalake_df[['spends', 'factor']]
run_time_df = pd.DataFrame(columns = ["Iteration", "Size", "Run_time"])
runs = 10

for i in range(runs):
    run_time_sum = 0

    for _ in range(runs):
        start = time.time()
        temp = new_df.apply(apply_func, axis = 1)
        end = time.time()

        run_time_sum += end-start

    run_time_df.loc[i] = [i+1, new_df.shape[0], run_time_sum/runs]
    new_df = pd.concat((new_df, new_df))

run_time_df

,Iteration,Size,Run_time
0,1.0,525.0,0.002975
1,2.0,1050.0,0.004342
2,3.0,2100.0,0.008492
3,4.0,4200.0,0.016587
4,5.0,8400.0,0.035308
5,6.0,16800.0,0.068936
6,7.0,33600.0,0.137443
7,8.0,67200.0,0.260515
8,9.0,134400.0,0.524878
9,10.0,268800.0,1.071239


In [7]:
datalake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         525 non-null    object 
 1   product      525 non-null    object 
 2   source       525 non-null    object 
 3   spends       525 non-null    int64  
 4   factor       525 non-null    float64
 5   received_at  525 non-null    int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 24.7+ KB


In [8]:
# Convert 'date' column to datetime
datalake_df['date'] = pd.to_datetime(datalake_df['date'], format='%Y-%m-%d')

# Set 'date' column as index
datalake_df.set_index('date', inplace=True)

# Convert 'product' and 'source' columns to categorical dtype
datalake_df['product'] = datalake_df['product'].astype('category')
datalake_df['source'] = datalake_df['source'].astype('category')

# Convert 'received_at' column to delta time
datalake_df['received_at'] = pd.to_timedelta(datalake_df['received_at'])

In [31]:
# Amount of memory used by the dataframe decreased because of the conversion of 'product' and 'source' columns to categorical dtype. Category dtype is a fixed-size datatype which is more memory efficient than object dtype because it stores the categories separately and uses integer values to represent the categories.
datalake_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 525 entries, 2024-01-01 to 2023-09-01
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype          
---  ------       --------------  -----          
 0   product      525 non-null    category       
 1   source       525 non-null    category       
 2   spends       525 non-null    int64          
 3   factor       525 non-null    float64        
 4   received_at  525 non-null    timedelta64[ns]
dtypes: category(2), float64(1), int64(1), timedelta64[ns](1)
memory usage: 18.2 KB


#### Indexing

In [108]:
datalake_df.loc['2024-12-01', 'product']

'product_1'

In [109]:
datalake_df.iloc[0, 0]

'product_1'

In [110]:
datalake_df['product'][0]

/var/folders/3y/__tmjg9s5wq_rd9fl2dtqqx80000gn/T/ipykernel_41495/4132577024.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  datalake_df['product'][0]


'product_1'

In [111]:
datalake_df['product']['2024-12-01']

'product_1'

#### Storing and Accessing Values

In [25]:
# When we are trying to access the values of the 'product' and 'factor' columns, we can see that the data type is 'object' even though we converted them.
datalake_df.values[:5, 0]

array(['product_1', 'product_1', 'product_1', 'product_1', 'product_1'],
      dtype=object)

In [26]:
datalake_df.values[:5, 3]

array([0.3238645161290322, 0.3369178571428571, 0.4398161290322581,
       0.2928354838709677, 0.2450548387096774], dtype=object)

In [42]:
# Now we can access the values of the 'product' and 'factor' columns differently and check the datetype.
datalake_df[['product']].values[:5].dtype

dtype('O')

In [44]:
datalake_df[['factor']].values[:5].dtype

dtype('float64')

In [46]:
# We can now try to replace some value in the 'factor' column using these methods.
print(datalake_df.values[0, 3])
datalake_df.values[0, 3] = 0.5
print(datalake_df.values[0, 3])

0.3238645161290322
0.3238645161290322


In [49]:
datalake_df['factor'].values[0] = 0.5
print(datalake_df['factor'].values[0])

0.5


In [56]:
only_float_df = pd.DataFrame({'A': [1.0, 2.0, 3.0, 4.0, 5.0], 
                              'B': [1.0, 2.0, 3.0, 4.0, 5.0]})

In [51]:
only_float_df

,A,B
0,1.0,1.0
1,2.0,2.0
2,3.0,3.0
3,4.0,4.0
4,5.0,5.0


In [57]:
only_float_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       5 non-null      float64
 1   B       5 non-null      float64
dtypes: float64(2)
memory usage: 212.0 bytes


In [55]:
print(only_float_df.values[0, 0])
only_float_df.values[0, 0] = 0.5
print(only_float_df.values[0, 0])

1.0
0.5


In [58]:
only_float_df['B'] = only_float_df['B'].astype('int8')

In [59]:
only_float_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A       5 non-null      float64
 1   B       5 non-null      int8   
dtypes: float64(1), int8(1)
memory usage: 177.0 bytes


In [60]:
print(only_float_df.values[0, 0])
only_float_df.values[0, 0] = 0.9
print(only_float_df.values[0, 0])

1.0
1.0


#### Iterating over DataFrame rows

In [65]:
type(datalake_df.iloc[0])

pandas.core.series.Series

In [63]:
# Iterating over DataFrame rows using iterrows() returns an iterator that yields index and row data as a Series which is mutable, but not recommended as there's no guarantee that replacement of values will work.
for row in datalake_df[:1].iterrows():
    print(row)
    print(type(row[1]))

(Timestamp('2024-01-01 00:00:00'), product                        product_1
source                source_1_product_1
spends                           1476304
factor                               0.5
received_at    0 days 00:28:53.098377843
Name: 2024-01-01 00:00:00, dtype: object)
<class 'pandas.core.series.Series'>


In [64]:
# Iterating over DataFrame rows using itertuples() returns an iterator that yields namedtuples of the values in each row, which is faster than iterrows() and is recommended for performance. Tuple values are immutable which is good, because pandas is not meant to be used for non-vectorized operations.
for row in datalake_df[:1].itertuples():
    print(row)
    print(type(row))

Pandas(Index=Timestamp('2024-01-01 00:00:00'), product='product_1', source='source_1_product_1', spends=1476304, factor=0.5, received_at=Timedelta('0 days 00:28:53.098377843'))
<class 'pandas.core.frame.Pandas'>


#### Add a new column and a new row in the DataFrame and check the time.

In [66]:
# Add a new column
start = time.time()
datalake_df['new_column'] = 0
end = time.time()
print(f"Time taken to add a new column: {end-start}")

Time taken to add a new column: 0.0003509521484375


In [69]:
# Add a new row
start = time.time()
datalake_df.loc[pd.to_datetime('2024-12-01'), :] = ['product_1', 'source_1_product_1', 1500, 0.5, pd.to_timedelta('00:00:00'), 0]
end = time.time()
print(f"Time taken to add a new row: {end-start}")

Time taken to add a new row: 0.0011162757873535156


#### Saving and Reading Data in CSV

In [73]:
# Save the dataframe to a new CSV file
start = time.time()
datalake_df.to_csv('datalake_df.csv', index=False)
end = time.time()
print(f"Time taken to save the dataframe: {end-start}")

Time taken to save the dataframe: 0.005197763442993164


In [74]:
# Show the size of the CSV file
!ls -lh datalake_df.csv

-rw-r--r--  1 nataliaziemba-jankowska  staff    43K Dec 11 04:45 datalake_df.csv


In [75]:
# Read the dataframe from the CSV file
start = time.time()
datalake_df_from_csv = pd.read_csv('datalake_df.csv')
end = time.time()
print(f"Time taken to read the dataframe: {end - start}")

Time taken to read the dataframe: 0.002218008041381836


In [12]:
datalake_df_from_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525 entries, 0 to 524
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   product      525 non-null    object 
 1   source       525 non-null    object 
 2   spends       525 non-null    int64  
 3   factor       525 non-null    float64
 4   received_at  525 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 20.6+ KB


#### Saving and Reading Data in Feather

In [14]:
# Save the dataframe to a feather file
start = time.time()
datalake_df.to_feather('datalake_df.feather')
end = time.time()
print(f"Time taken to save the dataframe: {end-start}")

Time taken to save the dataframe: 0.04310297966003418


In [18]:
# Show the size of the feather file
!ls -lh datalake_df.feather

-rw-r--r--  1 nataliaziemba-jankowska  staff    13K Dec 11 02:25 datalake_df.feather


In [15]:
# Read the dataframe from the feather file
start = time.time()
datalake_df_from_feather = pd.read_feather('datalake_df.feather')
end = time.time()
print(f"Time taken to read the dataframe: {end - start}")

Time taken to read the dataframe: 0.004857778549194336


In [16]:
datalake_df_from_feather.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 525 entries, 2024-01-01 to 2023-09-01
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype          
---  ------       --------------  -----          
 0   product      525 non-null    category       
 1   source       525 non-null    category       
 2   spends       525 non-null    int64          
 3   factor       525 non-null    float64        
 4   received_at  525 non-null    timedelta64[ns]
dtypes: category(2), float64(1), int64(1), timedelta64[ns](1)
memory usage: 18.2 KB


#### Saving and Reading Data in Parquet

In [19]:
# Save the dataframe to a parquet file
start = time.time()
datalake_df.to_parquet('datalake_df.parquet')
end = time.time()
print(f"Time taken to save the dataframe: {end-start}")

Time taken to save the dataframe: 0.029036998748779297


In [20]:
# Show the size of the parquet file
!ls -lh datalake_df.parquet

-rw-r--r--  1 nataliaziemba-jankowska  staff    12K Dec 11 02:43 datalake_df.parquet


In [21]:
# Read the dataframe from the parquet file
start = time.time()
datalake_df_from_parquet = pd.read_parquet('datalake_df.parquet')
end = time.time()
print(f"Time taken to read the dataframe: {end - start}")

Time taken to read the dataframe: 0.07878279685974121


In [22]:
datalake_df_from_parquet.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 525 entries, 2024-01-01 to 2023-09-01
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype          
---  ------       --------------  -----          
 0   product      525 non-null    category       
 1   source       525 non-null    category       
 2   spends       525 non-null    int64          
 3   factor       525 non-null    float64        
 4   received_at  525 non-null    timedelta64[ns]
dtypes: category(2), float64(1), int64(1), timedelta64[ns](1)
memory usage: 18.2 KB


##### There is no inherent multi-threading support available in Pandas so it will always stick to a single core utilization — leading to increased run-time, which is proportional to the size of the data.

### Groupby with Categorical dtype
When using a Categorical grouper (as a single grouper, or as part of multiple groupers), the observed keyword controls whether to return a cartesian product of all possible groupers values (observed=False) or only those that are observed groupers (observed=True).

In [82]:
datalake_df[:5]

,product,source,spends,factor,received_at,new_column
date,,,,,,
2024-01-01,product_1,source_1_product_1,1476304.0,0.500000,0 days 00:28:53.098377843,0.0
2023-02-01,product_1,source_2_product_1,2465434.0,0.336918,0 days 00:28:53.098377844,0.0
2024-07-01,product_1,source_1_product_1,1013569.0,0.439816,0 days 00:28:53.098377843,0.0
2024-03-01,product_1,source_1_product_1,1685347.0,0.292835,0 days 00:28:53.098377843,0.0
2024-05-01,product_1,source_2_product_1,4049280.0,0.245055,0 days 00:28:53.098377844,0.0


In [83]:
grouped_observed_false_df = datalake_df[:10].groupby(['product', 'source'])

/var/folders/3y/__tmjg9s5wq_rd9fl2dtqqx80000gn/T/ipykernel_41495/823797064.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_observed_false_df = datalake_df[:10].groupby(['product', 'source'])


In [84]:
grouped_observed_true_df = datalake_df[:10].groupby(['product', 'source'], observed=True)

In [85]:
# Calculate the sum of the 'spends' column for each group
start = time.time()
grouped_observed_false_df['spends'].sum()
end = time.time()
print(f"Time taken to calculate the sum of the 'spends' column for each group: {end-start}")

Time taken to calculate the sum of the 'spends' column for each group: 0.0013301372528076172


In [86]:
start = time.time()
grouped_observed_true_df['spends'].sum()
end = time.time()
print(f"Time taken to calculate the sum of the 'spends' column for each group: {end-start}")

Time taken to calculate the sum of the 'spends' column for each group: 0.0007140636444091797


In [95]:
# Create a DataFrame with a Categorical columns, 'A' and 'B' with 100 unique categories each and third column 'C' with random values
df = pd.DataFrame({'A': pd.Categorical([f'A{i}' for i in range(500)]*500),
                   'B': pd.Categorical([f'B{i}' for i in range(500)]*500),
                   'C': np.random.rand(250000)})

In [97]:
df.head()

,A,B,C
0,A0,B0,0.316452
1,A1,B1,0.919302
2,A2,B2,0.354673
3,A3,B3,0.737664
4,A4,B4,0.574394


In [98]:
grouped_observed_false_df = df[:10].groupby(['A', 'B'])

/var/folders/3y/__tmjg9s5wq_rd9fl2dtqqx80000gn/T/ipykernel_41495/3809492748.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_observed_false_df = df[:10].groupby(['A', 'B'])


In [99]:
grouped_observed_true_df = df[:10].groupby(['A', 'B'], observed=True)

In [101]:
# Calculate the sum of the 'C' column for each group
start = time.time()
grouped_observed_false_df['C'].sum()
end = time.time()
print(f"Time taken to calculate the sum of the 'C' column for each group: {end-start}")

Time taken to calculate the sum of the 'C' column for each group: 0.008615255355834961


In [100]:
start = time.time()
grouped_observed_true_df['C'].sum()
end = time.time()
print(f"Time taken to calculate the sum of the 'C' column for each group: {end-start}")

Time taken to calculate the sum of the 'C' column for each group: 0.0006411075592041016
